In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import json
from typing import List, Dict, Optional
import requests

In [ ]:
load_dotenv(override=True)

openrouter_api_key = os.getenv('OPENROUTER_API_KEY')
if openrouter_api_key:
    print(f"OpenRouter API Key exists and begins {openrouter_api_key[:8]}")
else:
    print("OpenRouter API Key (OPENROUTER_API_KEY) not set")

# Get Aviationstack API key
aviationstack_api_key = os.getenv('CLIENTSECRET')
if aviationstack_api_key:
    print(f"Aviationstack API Key exists and begins {aviationstack_api_key[:8]}")
else:
    print("Aviationstack API Key (CLIENTSECRET) not set")



In [ ]:
MODEL = "gpt-4o-mini"
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")
openai = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=openrouter_api_key,
)

system_message = """
You are a friendly and helpful AI flight booking assistant. Your primary role is to help users search for flights, check flight status, and find flight schedules using real-time flight data from Aviationstack API.

When users ask about flights, flight schedules, or flight status, you should proactively use the search_flights tool to retrieve accurate, up-to-date information. You can search by:
- Flight number (e.g., "AA100")
- Departure and arrival airports (using IATA codes like JFK, LAX, LHR)
- Flight date (YYYY-MM-DD format)
- Airline (using IATA codes like AA, DL, UA)

Provide clear, concise, and helpful responses. When presenting flight information, format it in an easy-to-read manner. If flight information is unavailable or cannot be found, clearly inform the user. Always use the search_flights tool when users ask flight-related questions rather than making assumptions.
"""


## Flight Search Tool

Create a tool to search for flights using the Aviationstack API

In [ ]:
# Function to search flights using Aviationstack API
def search_flights(flight_number: Optional[str] = None, 
                   dep_iata: Optional[str] = None,
                   arr_iata: Optional[str] = None,
                   flight_date: Optional[str] = None,
                   airline_iata: Optional[str] = None) -> str:
    """
    Search for flights using Aviationstack API.
    
    Args:
        flight_number: Flight number (e.g., 'AA100')
        dep_iata: Departure airport IATA code (e.g., 'JFK')
        arr_iata: Arrival airport IATA code (e.g., 'LAX')
        flight_date: Flight date in YYYY-MM-DD format
        airline_iata: Airline IATA code (e.g., 'AA')
    
    Returns:
        JSON string with flight information or error message
    """
    api_key = os.getenv('CLIENTSECRET')
    if not api_key:
        return "Error: Aviationstack API key (CLIENTSECRET) not found in environment variables"
    
    base_url = "https://api.aviationstack.com/v1/flights"
    params = {"access_key": api_key}
    
    # Add optional parameters
    if flight_number:
        params["flight_iata"] = flight_number
    if dep_iata:
        params["dep_iata"] = dep_iata.upper()
    if arr_iata:
        params["arr_iata"] = arr_iata.upper()
    if flight_date:
        params["flight_date"] = flight_date
    if airline_iata:
        params["airline_iata"] = airline_iata.upper()
    
    try:
        response = requests.get(base_url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        if data.get("error"):
            return f"API Error: {data.get('error', {}).get('info', 'Unknown error')}"
        
        flights = data.get("data", [])
        if not flights:
            return "No flights found matching the search criteria."
        
        # Format the response for better readability
        flight_info = []
        for flight in flights[:5]:  # Limit to first 5 results
            flight_data = flight.get("flight", {})
            departure = flight.get("departure", {})
            arrival = flight.get("arrival", {})
            airline = flight.get("airline", {})
            
            info = {
                "flight_number": flight_data.get("iata", "N/A"),
                "airline": airline.get("name", "N/A"),
                "departure": {
                    "airport": departure.get("airport", "N/A"),
                    "iata": departure.get("iata", "N/A"),
                    "scheduled": departure.get("scheduled", "N/A"),
                    "estimated": departure.get("estimated", "N/A"),
                    "status": departure.get("status", "N/A")
                },
                "arrival": {
                    "airport": arrival.get("airport", "N/A"),
                    "iata": arrival.get("iata", "N/A"),
                    "scheduled": arrival.get("scheduled", "N/A"),
                    "estimated": arrival.get("estimated", "N/A"),
                    "status": arrival.get("status", "N/A")
                },
                "flight_status": flight.get("flight_status", "N/A")
            }
            flight_info.append(info)
        
        return json.dumps(flight_info, indent=2)
    
    except requests.exceptions.RequestException as e:
        return f"Error fetching flight data: {str(e)}"
    except Exception as e:
        return f"Unexpected error: {str(e)}"

Cities: To get data about different cities, you can use the API's Cities endpoint. Only paid plan subscribers can use the search request parameter to receive airplane autocomplete suggestions.

In [ ]:
def get_cities(search: Optional[str] = None) -> str:
    
    print("cities called")
    access_key = os.getenv("CLIENTSECRET")
    if not access_key:
        return "Error: Aviationstack API key (CLIENTSECRET) not found in environment variables"
    
    base_url = "https://api.aviationstack.com/v1/cities"
    params = {"access_key": access_key}
    if search:
        params["search"] = search
    
    try:
        response = requests.get(base_url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        print(data)

        if data.get("error"):
            return f"API Error: {data.get('error', {}).get('info', 'Unknown error')}"
        
        cities = data.get("data", [])
        if not cities:
            return "No cities found matching the search criteria."
        
        city_info = []
        for city in cities:
            city_info.append({
                "city_name": city.get("city_name", "N/A"),
                "timezone": city.get("timezone", "N/A"),
                "iata": city.get("iata_code", "N/A"),
                "country_iso2": city.get("country_iso2", "N/A")
            })
        return json.dumps(city_info, indent=2)
    
    except requests.exceptions.RequestException as e:
        return f"Error fetching city data: {str(e)}"
    except Exception as e:
        return f"Unexpected error: {str(e)}"
            
    

In [ ]:
# Tool function definition for OpenAI
search_flights_function = {
    "type": "function",
    "function": {
        "name": "search_flights",
        "description": "Search for real-time and historical flight information using Aviationstack API. "
                       "Use this when users ask about flight status, flight schedules, or want to search for flights. "
                       "You can search by flight number, departure/arrival airports, flight date, or airline.",
        "parameters": {
            "type": "object",
            "properties": {
                "flight_number": {
                    "type": "string",
                    "description": "Flight number in IATA format (e.g., 'AA100', 'DL200'). Leave empty if searching by route."
                },
                "dep_iata": {
                    "type": "string",
                    "description": "Departure airport IATA code (e.g., 'JFK', 'LAX', 'LHR'). Use 3-letter airport codes."
                },
                "arr_iata": {
                    "type": "string",
                    "description": "Arrival airport IATA code (e.g., 'JFK', 'LAX', 'LHR'). Use 3-letter airport codes."
                },
                "flight_date": {
                    "type": "string",
                    "description": "Flight date in YYYY-MM-DD format (e.g., '2026-01-29'). Optional, defaults to today if not specified."
                },
                "airline_iata": {
                    "type": "string",
                    "description": "Airline IATA code (e.g., 'AA' for American Airlines, 'DL' for Delta). Optional."
                }
            },
            "required": [],
            "additionalProperties": False
        }
    }
}

search_cities_function = {
    "type": "function",
    "function": {
        "name": "get_cities",
        "description": "Get list of cities from Aviationstack API. Use when users need city names, IATA codes, timezones, or country info. Optional search term filters results (paid plan for autocomplete).",
        "parameters": {
            "type": "object",
            "properties": {
                "search": {
                    "type": "string",
                    "description": "Optional search term to filter cities (e.g., city name or partial match). Leave empty to get cities without filtering."
                }
            },
            "required": [],
            "additionalProperties": False
        }
    }
}


## Chat Function with Tool Calling

In [ ]:
def handle_tool_calls(message):
    """Handle tool calls from OpenAI"""
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "search_flights":
            arguments = json.loads(tool_call.function.arguments)
            flight_info = search_flights(
                flight_number=arguments.get("flight_number"),
                dep_iata=arguments.get("dep_iata"),
                arr_iata=arguments.get("arr_iata"),
                flight_date=arguments.get("flight_date"),
                airline_iata=arguments.get("airline_iata")
            )
            responses.append({
                "role": "tool",
                "content": flight_info,
                "tool_call_id": tool_call.id
            })
        elif tool_call.function.name == "get_cities":
            arguments = json.loads(tool_call.function.arguments)
            flight_info = search_flights(
                flight_number=arguments.get("flight_number"),
                dep_iata=arguments.get("dep_iata"),
                arr_iata=arguments.get("arr_iata"),
                flight_date=arguments.get("flight_date"),
                airline_iata=arguments.get("airline_iata")
            )
            responses.append({
                "role": "tool",
                "content": flight_info,
                "tool_call_id": tool_call.id
            })
    return responses

def chat(message, history):
    """Chat function for Gradio interface with tool calling support"""
    # Build conversation history
    messages = [{"role": "system", "content": system_message}]
    
    # Add conversation history - handle Gradio's format
    # History is a list of tuples: [(user_msg1, assistant_msg1), (user_msg2, assistant_msg2), ...]
    if history:
        for entry in history:
            # Handle different possible formats
            if isinstance(entry, (list, tuple)) and len(entry) >= 2:
                user_msg = entry[0]
                assistant_msg = entry[1]
                if user_msg:
                    messages.append({"role": "user", "content": str(user_msg)})
                if assistant_msg:
                    messages.append({"role": "assistant", "content": str(assistant_msg)})
            elif isinstance(entry, dict):
                # Handle dict format if Gradio uses it
                if "user" in entry:
                    messages.append({"role": "user", "content": str(entry["user"])})
                if "assistant" in entry:
                    messages.append({"role": "assistant", "content": str(entry["assistant"])})
    
    # Add current user message
    messages.append({"role": "user", "content": str(message)})
    
    # Call OpenAI with tool support
    max_iterations = 5  # Prevent infinite loops
    iteration = 0
    assistant_message = None
    
    while iteration < max_iterations:
        response = openai.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=[search_flights_function,search_cities_function],
            tool_choice="auto"
        )
        
        assistant_message = response.choices[0].message
        
        # Convert assistant_message to dict format for messages list
        assistant_dict = {
            "role": assistant_message.role,
            "content": assistant_message.content
        }
        
        # Add tool_calls if present
        if assistant_message.tool_calls:
            assistant_dict["tool_calls"] = [
                {
                    "id": tc.id,
                    "type": tc.type,
                    "function": {
                        "name": tc.function.name,
                        "arguments": tc.function.arguments
                    }
                }
                for tc in assistant_message.tool_calls
            ]
        
        messages.append(assistant_dict)
        
        # Check if tool calls are needed
        if assistant_message.tool_calls:
            # Handle tool calls
            tool_responses = handle_tool_calls(assistant_message)
            messages.extend(tool_responses)
            iteration += 1
        else:
            # Return the final response
            return assistant_message.content or ""
    
    # If we've exhausted iterations, return the last response
    return assistant_message.content if assistant_message and assistant_message.content else "I apologize, but I'm having trouble processing your request. Please try again."

In [ ]:
# Launch Gradio interface
gr.ChatInterface(fn=chat).launch()